In [1]:
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
import pandas as pd
import math
import numpy as np
import pickle
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import mean_squared_error
# !pip install shap
# import shap
from sklearn.preprocessing import StandardScaler, Normalizer
import matplotlib.pyplot as plt

2022-09-23 18:59:53.502809: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-23 18:59:53.507555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/anaconda3-2021-05/lib:/cm/shared/apps/pbspro-ce/19.1.3/lib/
2022-09-23 18:59:53.507573: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def reshape_sliding1(X, num_steps_x = 2, num_steps_y = 2):
    X = pd.DataFrame(X)
    X_transformed = [np.array(X.shift(i)) for i in range(num_steps_x + num_steps_y)]
    X_transformed = np.dstack(X_transformed)
    
    # swap time steps and dimensionality axes
    X_transformed = np.swapaxes(X_transformed, 1, 2)
    # flip time steps axis
    X_transformed = np.flip(X_transformed, 1)
    X_transformed = X_transformed[(num_steps_x+num_steps_y - 1):]
    return X_transformed

In [3]:
def load_EDdata(input_data_shiftedy,var,n_steps_x,n_steps_y):

    # number of time steps for sequence learning
    # n_steps_x = 14
    # number of time steps for sequence prediction
    # n_steps_y = 1

    split_at = int(len(input_data_shiftedy) * .8)
    val_split_at = int(len(input_data_shiftedy) * .9)


    scaler = StandardScaler()
    # if only prevalence data
    input_data_shiftedy = input_data_shiftedy.values.reshape((input_data_shiftedy.shape[0], -1))
    input_data_shiftedy[:(split_at + n_steps_x + n_steps_y)] = scaler.fit_transform(
        input_data_shiftedy[:(split_at + n_steps_x + n_steps_y)])
    # only transform valid and testing sets
    input_data_shiftedy[(split_at + n_steps_x + n_steps_y):] = scaler.transform(
        input_data_shiftedy[(split_at + n_steps_x + n_steps_y):])


    input_data_shiftedy = reshape_sliding1(input_data_shiftedy, 
                                           num_steps_x = n_steps_x,
                                          num_steps_y = n_steps_y)

    # train_test_split 

    X_train = input_data_shiftedy[:split_at, :n_steps_x, :data.shape[1]-1]
    X_valid = input_data_shiftedy[split_at:val_split_at, :n_steps_x, :data.shape[1]-1]
    X_test = input_data_shiftedy[val_split_at:, :n_steps_x, :data.shape[1]-1]

    Y = np.empty((input_data_shiftedy.shape[0], n_steps_x))
    for step_ahead in range(1, n_steps_y + 1):
        # print(step_ahead, step_ahead + n_steps_x)
        Y = input_data_shiftedy[..., step_ahead:step_ahead + n_steps_x, 0]
    Y_train = Y[:split_at, data.shape[1]-1:]
    Y_valid = Y[split_at:val_split_at,data.shape[1]-1:]
    Y_test = Y[val_split_at:,data.shape[1]-1:]

    # checking dimensionality
    print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape,X_valid.shape, Y_valid.shape)
    # assert X_train.shape[1:] == X_valid.shape[1:]
    # assert Y_train.shape[1:] == Y_valid.shape[1:]

    return X_train,Y_train,X_test,Y_test,X_valid,Y_valid,var


In [4]:
n_steps_x=14
n_steps_y=1

with open("edmond_datasets.pickle", "rb") as handle:
    input_data_shiftedy = pickle.load(handle)
# select only n, daily asthma visits and total, daily total AE visits, 
# comment out if including air pollution station data
first_col = input_data_shiftedy.n / input_data_shiftedy.total * 1000
input_data_shiftedy.insert(0, "prevalence", first_col)
input_data_shiftedy = input_data_shiftedy.drop(["n", "total", "y"], axis = 1)
data=pd.DataFrame(input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1),columns=['CO'])
data['NO2']=input_data_shiftedy.filter(like='NO2_', axis=1).mean(axis=1)
data['O3']=input_data_shiftedy.filter(like='O3_', axis=1).mean(axis=1)
data['SO2']=input_data_shiftedy.filter(like='SO2_', axis=1).mean(axis=1)
data['CO']=input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1)
data['FSP']=input_data_shiftedy.filter(like='FSP_', axis=1).mean(axis=1)
data[['prevalence','temp','RelHum']]=input_data_shiftedy[['prevalence','Mean (deg. C)','Mean Relative Humidity (%)']]
# input_data_shiftedy = pd.concat([input_data_shiftedy.n.reset_index(drop = True), 
#                         input_data_shiftedy.total.reset_index(drop = True)], axis = 1)
# input_data_shiftedy = input_data_shiftedy[var]
first_column = data.pop('prevalence')
data.insert(0, 'prevalence', first_column)

# if i=='prev':
#     data=data[['prevalence']]
# elif i=='ae':
#     data=data[['prevalence','temp','RelHum']]
# elif i =='poll':
input_data_shiftedy=data[['prevalence','FSP','O3','NO2','SO2','CO']]

split_at = int(len(input_data_shiftedy) * .8)
val_split_at = int(len(input_data_shiftedy) * .9)


scaler = StandardScaler()
# if only prevalence data
input_data_shiftedy = input_data_shiftedy.values.reshape((input_data_shiftedy.shape[0], -1))
input_data_shiftedy[:(split_at + n_steps_x + n_steps_y)] = scaler.fit_transform(
    input_data_shiftedy[:(split_at + n_steps_x + n_steps_y)])
# only transform valid and testing sets
input_data_shiftedy[(split_at + n_steps_x + n_steps_y):] = scaler.transform(
    input_data_shiftedy[(split_at + n_steps_x + n_steps_y):])


input_data_shiftedy = reshape_sliding1(input_data_shiftedy, 
                                       num_steps_x = n_steps_x,
                                      num_steps_y = n_steps_y)

data=input_data_shiftedy

X_train = data[:split_at, :n_steps_x, :data.shape[1]-1]
X_valid = input_data_shiftedy[split_at:val_split_at, :n_steps_x, :data.shape[1]-1]
X_test = input_data_shiftedy[val_split_at:, :n_steps_x, :data.shape[1]-1]

Y = np.empty((input_data_shiftedy.shape[0], n_steps_x))
for step_ahead in range(1, n_steps_y + 1):
    # print(step_ahead, step_ahead + n_steps_x)
    Y = input_data_shiftedy[..., step_ahead:step_ahead + n_steps_x, 0]
Y_train = Y[:split_at, data.shape[1]-1:]
Y_valid = Y[split_at:val_split_at,data.shape[1]-1:]
Y_test = Y[val_split_at:,data.shape[1]-1:]

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape,X_valid.shape, Y_valid.shape)


(1861, 14, 6) (1861, 0) (219, 14, 6) (219, 0) (233, 14, 6) (233, 0)


In [8]:
def run_EDmdl(X_train,Y_train,X_test,Y_test,X_valid,Y_valid):#,var):
    optimizer = keras.optimizers.Adam(clipvalue = 1)
    # simplified wavenet
    def last_time_step_mse(Y_true, Y_pred):
        return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = [X_train.shape[1], X_train.shape[2]]))
    for rate in (1, 2, 4, 8, 16) * 2:
        model.add(keras.layers.Conv1D(filters = 100, kernel_size = 2, padding = "causal",
                                     activation = "relu", dilation_rate = rate))
    model.add(keras.layers.Conv1D(filters = 1, kernel_size = 1))
    # model.add(keras.layers.LSTM(100))
    # model.add(keras.layers.Dense(1))
    model.add(keras.layers.Lambda(lambda x: tf.reshape(x, [-1, 14])))
    model.add(keras.layers.Dense(1)) ## to collapse all var to predict prevalence alone
    model.compile(loss = "mse", optimizer = optimizer, metrics = [last_time_step_mse])

#     history = model.fit(X_train, Y_train, epochs=30,verbose=0, shuffle=True, validation_data=(X_valid, Y_valid),callbacks = [callback])

#     pred = model.predict(X_test)
#     keras.backend.clear_session()
#     # mse = (mean_squared_error(pred, Y_test))
#     print('Test MSE: %.3f' % mse)

#     plt.plot(np.double(pred[:, -1]-.5).flatten(), alpha = .5,label='pred')
#     plt.plot(np.double(Y_test[:, -1]).flatten(), alpha = .5,label='valid')
#     # plt.title(var+'_MSE: '+str(mse))
#     plt.legend()
#     plt.savefig('edNNtest/'+var+'_mse.png')
    # print(model.summary())
#     plt.clf()
    return model

In [6]:
def load_model(X_train,Y_train,X_test,Y_test,X_valid,Y_valid):
    # use simple CNN structure
    in_shape = ([X_train.shape[1], X_train.shape[2]])
    model = keras.models.Sequential()
    model.add(keras.layers.ConvLSTM2D(32, kernel_size=(7, 7), padding='valid', return_sequences=True, input_shape=in_shape))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(keras.layers.ConvLSTM2D(64, kernel_size=(5, 5), padding='valid', return_sequences=True))
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(keras.layers.ConvLSTM2D(96, kernel_size=(3, 3), padding='valid', return_sequences=True))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.ConvLSTM2D(96, kernel_size=(3, 3), padding='valid', return_sequences=True))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.ConvLSTM2D(96, kernel_size=(3, 3), padding='valid', return_sequences=True))
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(keras.layers.Dense(320))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dropout(0.5))

    # out_shape = model.output_shape
    # print('====Model shape: ', out_shape)
    # model.add(Reshape((SequenceLength, out_shape[2] * out_shape[3] * out_shape[4])))
    model.add(keras.layers.LSTM(64, return_sequences=False))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(N_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    # model structure summary
    print(model.summary())

    return model

In [10]:
var_list = []
mse_list = []
# site_list = []
order_list = []
x_days=14
y_days=1

for i in ['all','ae','poll','prev']:
    # for SS in ['orig','shuf']:

    with open("edmond_datasets.pickle", "rb") as handle:
        input_data_shiftedy = pickle.load(handle)
    # select only n, daily asthma visits and total, daily total AE visits, 
    # comment out if including air pollution station data
    first_col = input_data_shiftedy.n / input_data_shiftedy.total * 1000
    input_data_shiftedy.insert(0, "prevalence", first_col)
    input_data_shiftedy = input_data_shiftedy.drop(["n", "total", "y"], axis = 1)
    data=pd.DataFrame(input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1),columns=['CO'])
    data['NO2']=input_data_shiftedy.filter(like='NO2_', axis=1).mean(axis=1)
    data['O3']=input_data_shiftedy.filter(like='O3_', axis=1).mean(axis=1)
    data['SO2']=input_data_shiftedy.filter(like='SO2_', axis=1).mean(axis=1)
    data['CO']=input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1)
    data['FSP']=input_data_shiftedy.filter(like='FSP_', axis=1).mean(axis=1)
    data[['prevalence','temp','RelHum']]=input_data_shiftedy[['prevalence','Mean (deg. C)','Mean Relative Humidity (%)']]
    # input_data_shiftedy = pd.concat([input_data_shiftedy.n.reset_index(drop = True), 
    #                         input_data_shiftedy.total.reset_index(drop = True)], axis = 1)
    # input_data_shiftedy = input_data_shiftedy[var]
    first_column = data.pop('prevalence')
    data.insert(0, 'prevalence', first_column)
    
    # dataset=data
    # values = dataset.values
    # groups = [0, 1, 2, 3, 4, 5, 6, 7]
    # i = 1
    # # plot each column
    # plt.figure(figsize=(10,8))
    # for group in groups:
    #     plt.subplot(len(groups), 1, i)
    #     plt.plot(values[:, group])
    #     plt.title(dataset.columns[group], y=0.5, loc='right')
    #     i += 1
    # plt.show()
    
    if i=='prev':
        data=data[['prevalence']]
    elif i=='ae':
        data=data[['prevalence','temp','RelHum']]
    elif i =='poll':
        data=data[['prevalence','FSP','O3','NO2','SO2','CO']]
    elif i =='all':
        data=data
    # n_days=7
    if keras.Model:
        keras.backend.clear_session()
    train_X, train_Y,test_X,test_Y,valid_X,valid_Y,var=load_EDdata(data,i,x_days,y_days)
    model=run_EDmdl(train_X, train_Y,test_X,test_Y,valid_X,valid_Y)#,var)
    # model=load_model(train_X, train_Y,test_X,test_Y,valid_X,valid_Y)#,var)
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    history = model.fit(X_train, Y_train, epochs=30,verbose=0, shuffle=True, validation_data=(X_valid, Y_valid),callbacks = [callback])

    pred = model.predict(X_test)
    # keras.backend.clear_session()
    # mse = (mean_squared_error(pred, Y_test))
    print('Test MSE: %.3f' % mse)

    plt.plot(np.double(pred[:, -1]-.5).flatten(), alpha = .5,label='pred')
    plt.plot(np.double(Y_test[:, -1]).flatten(), alpha = .5,label='valid')
    # plt.title(var+'_MSE: '+str(mse))
    plt.legend()
    plt.savefig('edNNtest/'+var+'_mse.png')
    # plt.clf()
    var_list.append(i)
    mse_list.append(mse)

df = pd.DataFrame(list(zip(var_list , mse_list)),#,order_list)), 
           columns =['var', ',mse'])

df.to_csv('NNtest/NN_mse.txt',sep='\t')

(1861, 14, 7) (1861, 7) (219, 14, 7) (219, 7) (233, 14, 7) (233, 7)


ValueError: in user code:

    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/dcmorgan/.conda/envs/mypy38/lib/python3.8/site-packages/keras/e

In [ ]:
data

In [ ]:
with open("edmond_datasets.pickle", "rb") as handle:
    input_data_shiftedy = pickle.load(handle)

first_col = input_data_shiftedy.n / input_data_shiftedy.total * 1000
input_data_shiftedy.insert(0, "prevalence", first_col)
input_data_shiftedy = input_data_shiftedy.drop(["n", "total", "y"], axis = 1)

In [ ]:
data=pd.DataFrame(input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1),columns=['CO'])
data['NO2']=input_data_shiftedy.filter(like='NO2_', axis=1).mean(axis=1)
data['O3']=input_data_shiftedy.filter(like='O3_', axis=1).mean(axis=1)
data['SO2']=input_data_shiftedy.filter(like='SO2_', axis=1).mean(axis=1)
data['CO']=input_data_shiftedy.filter(like='CO_', axis=1).mean(axis=1)
data['FSP']=input_data_shiftedy.filter(like='FSP_', axis=1).mean(axis=1)
data[['prevalence','temp','RelHum']]=input_data_shiftedy[['prevalence','Mean (deg. C)','Mean Relative Humidity (%)']]

In [ ]:
data

# modelling
1. as of 23rd aug 2022: rnn loss nan problem <br>
possible remedies: normalize data, regularization, increase batch size <br>
https://datascience.stackexchange.com/questions/68331/keras-sequential-model-returns-loss-nan <br>
2. as of 24th aug 2022: try to add overall AE visit to include hospital avoidance effect
3. as of 25th aug 2022: corrected reshaping problem, performance is still shit, try remove air pollution, try larger learning rate
4. as of 26th aug 2022: try seq2seq model, it worked 
5. from the results it seems most contributing factor is total AE trend 


# shap value

In [ ]:
def f(X):
    return model.predict(X)[:, -1]

In [ ]:
explainer = shap.DeepExplainer(model, X_test)

In [ ]:
X_test[:50, -1, :].shape

In [ ]:
model.inputs

In [ ]:
# https://github.com/slundberg/shap/issues/1226 the nonetype shape problem

In [ ]:
shap_values = explainer.shap_values(X_test)

In [ ]:
X_test.shape